<a href="https://colab.research.google.com/github/hoon-i/ML-with-Math/blob/main/LangChain_PromptTemplate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q openai langchain langchain-openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.0/974.0 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.7/314.7 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.9/124.9 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 19.3 MB/s eta 0:00:00


In [3]:
!pip install langchain_community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 7.5 MB/s eta 0:00:00


In [4]:
#API KEY 저장을 위한 os 라이브러리 호출
import os

#기본 LLM 로드를 위한 라이브러리 호출
from langchain.llms import OpenAI

#채팅 LLM 로드를 위한 라이브러리 호출
from langchain.chat_models import ChatOpenAI

In [5]:
#OPENAI API키 저장
os.environ["OPENAI_API_KEY"] = 'API'

-------

#### Davinch-003 모델 설정하기

In [6]:
davinch3 = OpenAI(
    model_name="gpt-3.5-turbo-instruct",
    max_tokens = 1000
)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


#### (1) 프롬프트 템플릿 맛보기

프롬프트 템플릿은 크게 2가지가 존재합니다.
1. Prompt Template
2. Chat Prompt Template

1번 Prompt Template은 일반적인 프롬프트 템플릿을 생성할때 활용합니다.

2번 Chat Prompt Template은 채팅 LLM에 프롬프트를 전달하는 데에 활용할 수 있는 특화 프롬프트 템플릿입니다.



In [7]:
from langchain.prompts import PromptTemplate, ChatPromptTemplate

#프롬프트 템플릿을 통해 매개변수 삽입 가능한 문자열로 변환
string_prompt = PromptTemplate.from_template("tell me a joke about {subject}")

#매개변수 삽입한 결과를 string_prompt_value에 할당
string_prompt_value = string_prompt.format_prompt(subject="soccer")

#채팅LLM이 아닌 LLM과 대화할 때 필요한 프롬프트 = string prompt
string_prompt_value

StringPromptValue(text='tell me a joke about soccer')

In [8]:
#to_string() 함수를 통해 prompt template으로 생성한 문장 raw_text 반환 가능
print(string_prompt_value.to_string())

tell me a joke about soccer


In [9]:
chat_prompt = ChatPromptTemplate.from_template("tell me a joke about {subject}")
chat_prompt_value = chat_prompt.format_prompt(subject="soccer")
chat_prompt_value

ChatPromptValue(messages=[HumanMessage(content='tell me a joke about soccer')])

In [10]:
chat_prompt_value.to_string()

'Human: tell me a joke about soccer'

#### (2) 프롬프트 템플릿 활용해보기

반복적인 프롬프트를 삽입해야하는 경우, Prompt Template를 통해 간편하게 LLM을 활용할 수 있습니다.

- GPT-3와 프롬프트 템플릿을 활용하여 대화해보기

In [11]:
from langchain.prompts.prompt import PromptTemplate

template = """
너는 요리사야. 내가 가진 재료들을 갖고 만들 수 있는 요리를 추천하고, 그 요리의 레시피를 제시해줘.
내가 가진 재료는 아래와 같아.

<재료>
{재료}

"""
prompt_template = PromptTemplate(
    input_variables = ['재료'],
    template = template
)


In [12]:
print(prompt_template.format(재료 = '양파, 계란, 사과, 빵'))


너는 요리사야. 내가 가진 재료들을 갖고 만들 수 있는 요리를 추천하고, 그 요리의 레시피를 제시해줘.
내가 가진 재료는 아래와 같아.

<재료>
양파, 계란, 사과, 빵




In [13]:
print(davinch3(
    prompt_template.format(
        재료 = '양파, 계란, 사과, 빵'
    )
))

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


그렇다면 양파와 계란으로 만들 수 있는 간단한 요리로는 계란말이나 양파전이 있겠네요. 빵과 사과로는 샌드위치나 사과파이를 만들 수 있을 것 같습니다. 각각의 레시피는 아래와 같습니다.

<계란말이>

1. 양파를 적당한 크기로 썰어줍니다.
2. 계란 2개를 그릇에 넣고 잘 풀어줍니다.
3. 팬에 기름을 두르고 양파를 넣어서 볶다가 계란을 부어줍니다.
4. 접시에 담아서 소금과 후추로 간을 해주고 마무리합니다.

<양파전>

1. 양파를 적당한 크기로 썰어줍니다.
2. 밀가루 1컵에 물을 조금씩 넣어가면서 반죽을 만듭니다.
3. 팬에 기름을 두르고 반죽을 부어줍니다.
4. 반죽 위에 양파를 넣어줍니다.
5. 접시에 담아서 간장과 식초를 섞어서 만든 된장소스와 함께 즐겨주세요.

<샌드위치>

1. 빵을 2장 준비합니다.
2. 빵에 마요네즈나 버터를 바르고, 그 위에 사과와 양파를 올려줍니다.
3. 다시 빵을 한 장 더 올려서 뚜껑을 만들어줍니다.
4. 샌드위치 그릴이나 냄비에 기름을 두르고 샌드위치를 구워줍니다.
5. 녹은 치즈나 토마토소스 등을 추가하면 더 맛있게 즐길 수 있습니다.

<사과파이>

1. 빵을 적당한 크기로 썰어줍니다.
2. 사과를 적당한 크기로 썰어줍니다.
3. 팬에 기름을 두르고 빵과 사과를 넣어서 볶아줍니다.
4. 사과가 부드러워질 때까지 볶아준 후, 새로운 접시에 옮겨서 식힙니다.
5. 냉동파이 또는 레디메이드 파이 크러스트에 볶은 빵과 사과를 올려줍니다.
6. 오븐에서 15분 정도 구워주면 완성입니다. 


-  ChatGPT와 프롬프트 템플릿을 활용하여 대화해보기

In [14]:
from langchain.prompts import (
    ChatPromptTemplate,
    PromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

In [15]:
# ChatGPT 모델을 로드합니다.
chatgpt = ChatOpenAI(temperature=0)

#ChatGPT에게 역할을 부여합니다.(위에서 정의한 Template 사용)
system_message_prompt = SystemMessagePromptTemplate.from_template(template)

#사용자가 입력할 매개변수 template을 선언합니다.
human_template = "{재료}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

#ChatPromptTemplate에 system message와 human message 템플릿을 삽입합니다.
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

#ChatGPT API에 ChatPromptTemplate을 입력할 때, human message의 매개변수인 '재료'를 할당하여 전달합니다.
#이와 같은 방식을 통해 ChatGPT는 ChatPromptTemplate의 구성요소인 system message와 human message를 전달받아, 대답 생성에 활용합니다.
answer = chatgpt(chat_prompt.format_prompt(재료="양파, 계란, 사과, 빵").to_messages())
print(answer.content)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


제가 추천하는 요리는 "양파 계란 토스트"입니다. 이 요리는 간단하면서도 맛있는 아침 식사나 간식으로 좋습니다. 아래는 양파 계란 토스트의 레시피입니다.

**양파 계란 토스트 레시피:**

**재료:**
- 빵 4조각
- 양파 1개
- 계란 2개
- 소금
- 후추
- 식용유

**만드는 법:**
1. 양파를 깍둑썰기로 썰어줍니다.
2. 팬에 식용유를 두르고 양파를 볶아 투명해질 때까지 볶아줍니다.
3. 볶은 양파를 그릇에 옮기고 계란 2개를 넣어 풀어줍니다. 소금과 후추로 간을 해줍니다.
4. 빵을 굽기 전에 계란을 골고루 섞어줍니다.
5. 팬에 식용유를 두르고 계란을 부어 반죽이 굳어질 때까지 익혀줍니다.
6. 계란지단을 빵 한 조각에 올려준 후 다른 빵으로 덮어줍니다.
7. 겉면이 바삭해질 때까지 노릇노릇하게 구워주면 완성입니다.

양파 계란 토스트는 부드러운 계란과 고소한 양파의 조화가 맛있는 요리입니다. 맛있게 즐겨보세요!


###(3) Few-shot 예제를 통한 프롬프트 템플릿

Few-shot이란, 딥러닝 모델이 결과물을 출력할 때 예시 결과물을 제시함으로써 원하는 결과물로 유도하는 방법론입니다.

LLM 역시, Few-shot 예제를 제공하면 예제와 유사한 형태의 결과물을 출력합니다.

내가 원하는 결과물의 형태가 특수하거나, 구조화된 답변을 원할 경우, 결과물의 예시를 수 개 제시함으로써 결과물의 품질을 향상시킬 수 있습니다.

In [16]:
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts.prompt import PromptTemplate

examples = [
  {
    "question": "아이유로 삼행시 만들어줘",
    "answer":
"""
아: 아이유는
이: 이런 강의를 들을 이
유: 유가 없다.
"""
  },

  {
    "question": "김민수로 삼행시 만들어줘",
    "answer":
"""
김: 김치는 맛있다
민: 민달팽이도 좋아하는 김치!
수: 수억을 줘도 김치는 내꺼!
"""
  }
]

In [17]:
example_prompt = PromptTemplate(input_variables=["question", "answer"], template="Question: {question}\n{answer}")

print(example_prompt.format(**examples[0]))

Question: 아이유로 삼행시 만들어줘

아: 아이유는
이: 이런 강의를 들을 이
유: 유가 없다.



In [18]:
prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="Question: {input}",
    input_variables=["input"]
)

print(prompt.format(input="호날두로 삼행시 만들어줘"))

Question: 아이유로 삼행시 만들어줘

아: 아이유는
이: 이런 강의를 들을 이
유: 유가 없다.


Question: 김민수로 삼행시 만들어줘

김: 김치는 맛있다
민: 민달팽이도 좋아하는 김치!
수: 수억을 줘도 김치는 내꺼!


Question: 호날두로 삼행시 만들어줘


In [20]:
print(davinch3.predict("호날두로 삼행시 만들어줘"))



"폴포갓 들고 뛰는 선수 날뛰는 날두
그 어둠을 밝히는 샛별 같은 날두
축구계의 전설, 판타지 속의 날두"


In [21]:
print(davinch3(
    prompt.format(input="호날두로 삼행시 만들어줘")
))



호: 호랑이는 무서운 동물
날: 날마다 먹이를 찾는다
두: 두려움 없는 호랑이는 없다


### (4) Example Selector를 이용한 동적 Few-shot 러닝

Few-shot 예제를 동적으로 입력하고 싶은 경우, Example Selector를 활용할 수 있습니다.

LLM이 여러 작업을 수행하도록 만들되 내가 원하는 범위의 대답을 출력하도록 하려면 사용자의 입력에 동적으로 반응해야 합니다.이와 동시에, 예제를 모두 학습시키는 것이 아니라 적절한 예시만 포함하도록 함으로써 입력 prompt의 길이를 제한하고, 이를 통해 오류가 발생하지 않도록 조절할 수 있습니다.

In [22]:
from langchain.prompts.example_selector import SemanticSimilarityExampleSelector
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import FewShotPromptTemplate, PromptTemplate

example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template="Input: {input}\nOutput: {output}",
)

# These are a lot of examples of a pretend task of creating antonyms.
examples = [
    {"input": "행복", "output": "슬픔"},
    {"input": "흥미", "output": "지루"},
    {"input": "불안", "output": "안정"},
    {"input": "긴 기차", "output": "짧은 기차"},
    {"input": "큰 공", "output": "작은 공"},
]

In [23]:
!pip install chromadb
!pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.8/526.8 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 41.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.9/59.9 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.0/107.0 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 11.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.7/283.7 kB 36.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB

In [24]:
example_selector = SemanticSimilarityExampleSelector.from_examples(
    # This is the list of examples available to select from.
    examples,
    # This is the embedding class used to produce embeddings which are used to measure semantic similarity.
    OpenAIEmbeddings(),
    # This is the VectorStore class that is used to store the embeddings and do a similarity search over.
    Chroma,
    # This is the number of examples to produce. 예시 개수
    k=1
)
similar_prompt = FewShotPromptTemplate(
    # We provide an ExampleSelector instead of examples.
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="주어진 입력에 대해 반대의 의미를 가진 단어를 출력해줘",
    suffix="Input: {단어}\nOutput:",
    input_variables=["단어"],
)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [25]:
# Input is a feeling, so should select the happy/sad example
print(similar_prompt.format(단어="무서운"))

주어진 입력에 대해 반대의 의미를 가진 단어를 출력해줘

Input: 불안
Output: 안정

Input: 무서운
Output:


In [26]:
# Input is a feeling, so should select the happy/sad example
print(similar_prompt.format(단어="큰 비행기"))

주어진 입력에 대해 반대의 의미를 가진 단어를 출력해줘

Input: 긴 기차
Output: 짧은 기차

Input: 큰 비행기
Output:


In [27]:
query = "큰 비행기"

print(davinch3(
    similar_prompt.format(단어=query)
))

 작은 비행기


### (5) Output Parser를 활용한 출력값 조정

LLM의 답변을 내가 원하는 형태로 고정하고 싶다면 OutputParser 함수를 활용할 수 있습니다. 리스트, JSON 형태 등 다양한 형식의 답변을 고정하여 출력할 수 있습니다.

In [28]:
from langchain.output_parsers import CommaSeparatedListOutputParser
from langchain.prompts import PromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI

In [29]:
output_parser = CommaSeparatedListOutputParser()

In [30]:
format_instructions = output_parser.get_format_instructions()
format_instructions

'Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`'

In [31]:
prompt = PromptTemplate(
    template="{주제} 5개를 추천해줘.\n{format_instructions}",
    input_variables=["주제"],
    partial_variables={"format_instructions": format_instructions}
)

In [32]:
model = OpenAI(temperature=0)

In [36]:
_input = prompt.format(주제="과일")
output = model(_input)

In [37]:
output

'\n\napple, banana, orange, strawberry, pineapple'

In [38]:
output_parser.parse(output)

['apple', 'banana', 'orange', 'strawberry', 'pineapple']